In [ ]:
pip install opencv-python pillow easyocr ultralytics deepface tf-keras

In [1]:
import json
import os
import pandas as pd
import isodate
import requests
import numpy as np
import cv2
from collections import Counter
from sklearn.cluster import KMeans
import easyocr
import torch
import webcolors
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import random

C:\Users\duwjd\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [115]:
with open('video_info.json', "r", encoding="utf-8") as file:
        data = json.load(file)

flattened_data = [video for sublist in data for video in sublist]
df = pd.DataFrame(flattened_data)

def convert_duration(iso_duration):
    if pd.isna(iso_duration):  # NaN 값 처리
        return "00:00:00"
    if not isinstance(iso_duration, str):  # 문자열이 아닐 경우 처리
        return "00:00:00"

    try:
        duration = isodate.parse_duration(iso_duration)
        total_seconds = int(duration.total_seconds())
        return f"{total_seconds // 3600:02}:{(total_seconds % 3600) // 60:02}:{total_seconds % 60:02}"
    except Exception as e:
        print(f"Error parsing duration: {iso_duration} -> {e}")
        return "00:00:00"

# duration 변환 적용
df['duration'] = df['duration'].apply(convert_duration)

# 1분 이상인 데이터 필터링
df = df[df['duration'] > "00:01:00"]

In [117]:
df=df.iloc[10:30,:]
df

,title,video_id,published_date,thumbnail_url,view_count,like_count,comment_count,category_id,duration,channel_id
19,"ENG│코스트코 고기 2.5키로 소분하는 자취 일상. 대파꼬치전, 또띠아와퍼 만들고...",uxDZyUHO9sE,2024-08-18,https://i.ytimg.com/vi/uxDZyUHO9sE/hqdefault.jpg,1113043.0,27876.0,597.0,22,00:33:51,"[planD플랜디, UCPaApoLkmPqxCeZF7gjoc3w]"
20,"＂이불을 한 채만 쓸까요?＂ 더 가까워진 임지연♥추영우, 찐-한 합방💞 | 옥씨부인...",n9TTmAATM1M,2025-01-19,https://i.ytimg.com/vi/n9TTmAATM1M/hqdefault.jpg,1975060.0,4139.0,235.0,24,00:02:54,"[JTBC Drama, UCkbJc8jMcTXwhtmN5VMwfXg]"
22,더 이상 물러날 곳❌ 의금부에 추포된 하율리-최정우 | 옥씨부인전 16회 | JTB...,6L66YeMkkLQ,2025-01-26,https://i.ytimg.com/vi/6L66YeMkkLQ/hqdefault.jpg,1248961.0,5547.0,425.0,24,00:02:56,"[JTBC Drama, UCkbJc8jMcTXwhtmN5VMwfXg]"
23,[스페셜 선공개] ＂현감 얼굴에 칼자국..＂ 임지연 향한 윤지혜의 의미심장한 질문....,Xu6MTsCNunA,2025-01-15,https://i.ytimg.com/vi/Xu6MTsCNunA/hqdefault.jpg,1093319.0,7734.0,461.0,24,00:02:36,"[JTBC Drama, UCkbJc8jMcTXwhtmN5VMwfXg]"
25,"그렇게 유난을 떨던 공개 장기연애 끝, 한달 뒤 떠난 도쿄여행에서 생긴 일💧",ehUnAK-uNqk,2025-01-12,https://i.ytimg.com/vi/ehUnAK-uNqk/hqdefault.jpg,341820.0,2864.0,129.0,22,00:46:40,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]"
26,"VLOG • 연휴동안 생긴 일✨ 자취요리, smp 리터치, 어색한 썸남보이랑 데이트...",ACdNDBZw_4c,2025-02-17,https://i.ytimg.com/vi/ACdNDBZw_4c/hqdefault.jpg,88353.0,898.0,114.0,22,00:12:53,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]"
27,"VLOG • 두피문신 smp 후기👩🏻‍🦲, 레전드 야장맛집 뿌시기, 행궁동과 홍대...",eZVo_G-S_5g,2024-10-12,https://i.ytimg.com/vi/eZVo_G-S_5g/hqdefault.jpg,84381.0,389.0,127.0,22,00:16:48,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]"
28,"VLOG • 그래요 나 연애해요,, 예뻐야 하는 날 야무지게 준비하고🧖🏻‍♀️ 남친...",qiuSUSiyGkI,2025-03-10,https://i.ytimg.com/vi/qiuSUSiyGkI/hqdefault.jpg,77976.0,1133.0,148.0,22,00:19:00,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]"
29,"VLOG • 2달 차 솔로 일상😇 기어코 이사, 대전여행, 청첩장모임, 집밥, 행궁...",WXgiwBUPhyc,2025-01-24,https://i.ytimg.com/vi/WXgiwBUPhyc/hqdefault.jpg,58725.0,525.0,110.0,22,00:20:57,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]"
35,"입냄새의 진짜 원인, 목에 낀 똥을 싹 없애는 간단한 방법 (이영섭 원장 2부)",dR5scpolbQY,2024-05-10,https://i.ytimg.com/vi/dR5scpolbQY/hqdefault.jpg,4179526.0,47740.0,999.0,26,00:12:08,"[지식인사이드, UCA_hgsFzmynpv1zkvA5A7jA]"


## 썸네일에서 피쳐 추출하기

In [119]:
# YOLO 모델 로드 (Ultralytics YOLOv5 예제)
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
ocr_reader = easyocr.Reader(['en', 'ko'])  # 영어 & 한글 OCR 지원

# 이미지 로드 (URL에서 다운로드)
def load_image_from_url(url):
    try:
        response = requests.get(url, stream=True, timeout=5)
        response.raise_for_status()
        image = np.asarray(bytearray(response.content), dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        if image is None:
            raise ValueError("이미지를 디코딩할 수 없습니다.")
        return image
    except requests.RequestException as e:
        print(f"이미지를 불러오는 중 오류 발생: {e}")
        return None

# 이미지 전처리 (그레이스케일 + 이진화)
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    denoised_image = cv2.GaussianBlur(binary_image, (5, 5), 0)
    return denoised_image

# 텍스트 분석 (OCR)
def extract_text(image, confidence_threshold=0.7):
    results = ocr_reader.readtext(image)
    text_info = []
    for (bbox, text, prob) in results:
        if prob >= confidence_threshold: 
            (top_left, _, bottom_right, _) = bbox
            x, y = int(top_left[0]), int(top_left[1])
            width = int(bottom_right[0] - top_left[0])
            height = int(bottom_right[1] - top_left[1])
            area = width * height
            
            text_info.append({
                "text": text,
                "x": x, "y": y,
                "width": width, "height": height,
                "area": area,
                "probability": prob 
            })
    return text_info

# 객체 탐지 (YOLO)
def detect_objects(image):
    if image is None:
        return {"objects": [], "central_focus": False}
    
    results = yolo_model(image)
    objects = []
    central_focus = False
    height, width, _ = image.shape
    
    for result in results.xyxy[0]:  # YOLOv5 결과
        x1, y1, x2, y2, conf, cls = map(int, result[:6])
        label = yolo_model.names[cls]
        
        # 객체의 중앙 여부 확인
        obj_center_x = (x1 + x2) / 2
        obj_center_y = (y1 + y2) / 2
        if (width * 0.3) < obj_center_x < (width * 0.7) and (height * 0.3) < obj_center_y < (height * 0.7):
            central_focus = True
        
        objects.append({"label": label, "x": x1, "y": y1, "width": x2-x1, "height": y2-y1})
    
    return {"objects": objects, "central_focus": central_focus}


# 주요 색상 추출 및 이름 변환
def closest_colour(requested_colour):
    """주어진 RGB 값과 가장 가까운 CSS3 색상명을 찾음"""
    min_colours = {}
    for name in webcolors.names("css3"):
        r_c, g_c, b_c = webcolors.name_to_rgb(name)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_color_name_from_rgb(r, g, b):
    """정확한 색상이 있으면 반환, 없으면 가장 가까운 색상명 반환"""
    try:
        return webcolors.rgb_to_name((r, g, b), spec='css3')
    except ValueError:
        return closest_colour((r, g, b))

def extract_colors(image, num_colors=5):
    """이미지에서 주요 색상을 추출하고, 각 색상의 비율을 반환"""
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).reshape((-1, 3))
    kmeans = KMeans(n_clusters=num_colors, n_init="auto")
    labels = kmeans.fit_predict(image)
    palette = kmeans.cluster_centers_.astype(int)
    
    # 색상의 출현 빈도 계산
    counter = np.bincount(labels)
    total_count = np.sum(counter)
    # 주요 색상 및 비율 저장
    color_ratios = {tuple(palette[i]): counter[i] / total_count for i in range(len(palette))}
    # 색상명을 가져오도록 수정
    color_list = [(get_color_name_from_rgb(*color), ratio) for color, ratio in color_ratios.items()]
    
    return sorted(color_list, key=lambda x: x[1], reverse=True)

# 🔹 밝기 및 대비 계산
def calculate_brightness(image):
    return np.mean(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

def calculate_contrast(image):
    return np.std(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

# 🔹 전체 썸네일 분석 함수
def analyze_image(image_url):
    image = load_image_from_url(image_url)
    if image is None:
        return None
    
    # Step 1: 이미지 특성 추출 (색상, 밝기, 대비)
    colors = extract_colors(image)
    brightness = calculate_brightness(image)
    contrast = calculate_contrast(image)
    
    # Step 2: 이미지 전처리 후 OCR 텍스트 추출
    processed_image = preprocess_image(image)
    text_data = extract_text(processed_image, confidence_threshold=0.7)
    
    # Step 3: 객체 탐지 (YOLO)
    object_data = detect_objects(image)
    
    # 결과 리턴
    result = {
        "dominant_colors": colors,
        "brightness": brightness,
        "contrast": contrast,
        "text_details": text_data,
        "object_details": object_data
    }
    
    return result

brightness_li=[]
contrast_li=[]
dominant_colors_li=[]
text_details_li=[]
largest_text_li=[]
objects_details_li=[]

for i in df['thumbnail_url']:
    image_url = i
    analysis_result = analyze_image(image_url)
    
    brightness_li.append(analysis_result['brightness'])
    contrast_li.append(analysis_result['contrast'])
    dominant_colors_li.append(analysis_result['dominant_colors'])
    text_details_li.append(analysis_result['text_details'])
    objects_details_li.append(analysis_result['object_details']['objects'])

df['brightness']=brightness_li
df['contrast']=contrast_li
df['dominant_colors']=dominant_colors_li
df['text_details']=text_details_li
df['objects_details']=objects_details_li

Using cache found in C:\Users\duwjd/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-18 Python-3.11.7 torch-2.6.0+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
C:\Users\duwjd/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\duwjd/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\duwjd/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', 

In [120]:
df.iloc[:,8:]

,duration,channel_id,brightness,contrast,dominant_colors,text_details,objects_details
19,00:33:51,"[planD플랜디, UCPaApoLkmPqxCeZF7gjoc3w]",121.515370,77.338087,"[(darkkhaki, 0.2679224537037037), (black, 0.25...",[],"[{'label': 'bottle', 'x': 21, 'y': 199, 'width..."
20,00:02:54,"[JTBC Drama, UCkbJc8jMcTXwhtmN5VMwfXg]",51.828316,54.035172,"[(darkslategrey, 0.43203125), (black, 0.251967...",[],[]
22,00:02:56,"[JTBC Drama, UCkbJc8jMcTXwhtmN5VMwfXg]",116.180718,89.289904,"[(black, 0.2563252314814815), (darkslategrey, ...",[],"[{'label': 'person', 'x': 199, 'y': 51, 'width..."
23,00:02:36,"[JTBC Drama, UCkbJc8jMcTXwhtmN5VMwfXg]",64.056331,64.310216,"[(darkslategrey, 0.33379050925925924), (black,...",[],"[{'label': 'person', 'x': 200, 'y': 55, 'width..."
25,00:46:40,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]",79.875556,63.603121,"[(black, 0.29823495370370373), (sienna, 0.2812...",[],"[{'label': 'person', 'x': 70, 'y': 45, 'width'..."
26,00:12:53,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]",52.929369,57.947802,"[(black, 0.563136574074074), (darkslategrey, 0...","[{'text': 'new!', 'x': 412, 'y': 70, 'width': ...","[{'label': 'person', 'x': 142, 'y': 62, 'width..."
27,00:16:48,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]",96.419520,80.071132,"[(black, 0.34930555555555554), (rosybrown, 0.2...",[],"[{'label': 'person', 'x': 240, 'y': 55, 'width..."
28,00:19:00,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]",74.573611,73.484784,"[(black, 0.42508101851851854), (dimgrey, 0.151...","[{'text': '브이로그;', 'x': 165, 'y': 252, 'width'...","[{'label': 'person', 'x': 158, 'y': 64, 'width..."
29,00:20:57,"[현주zutube, UCJMMSPO-0YnhxRWfGOPmqvA]",104.983252,73.115501,"[(black, 0.311875), (grey, 0.28423611111111113...",[],"[{'label': 'person', 'x': 146, 'y': 123, 'widt..."
35,00:12:08,"[지식인사이드, UCA_hgsFzmynpv1zkvA5A7jA]",46.880249,72.696192,"[(black, 0.674212962962963), (peru, 0.11410879...","[{'text': '(이비인무과 영업비일)', 'x': 10, 'y': 50, 'w...","[{'label': 'person', 'x': 250, 'y': 49, 'width..."


## 썸네일 분석결과 정리 및 칼럼 추출

In [197]:
url='https://i.ytimg.com/vi/FR4E9DPYG3A/hqdefault.jpg'
response = requests.get(url, stream=True, timeout=5)
response.raise_for_status()
image = np.asarray(bytearray(response.content), dtype=np.uint8)
image = cv2.imdecode(image, cv2.IMREAD_COLOR)
# 이미지 크기 얻기
height, width, _ = image.shape  # height와 width 값을 가져옵니다.
print(f"Image Width: {width}, Image Height: {height}")

Image Width: 480, Image Height: 360


In [123]:
def classify_positions(image_width, image_height, text_details, object_details):
    def classify_position(x, y, width, height):
        """ 위치를 왼쪽/중간/오른쪽, 위/중간/아래, 크기별로 분류 """
        right_x = x + width
        bottom_y = y + height

        # 가로 위치 (left, middle, right)
        if right_x < image_width * 0.33:
            horizontal_position = "left"
        elif x > image_width * 0.67:
            horizontal_position = "right"
        else:
            horizontal_position = "middle"

        # 세로 위치 (up, middle, down)
        if bottom_y < image_height * 0.33:
            vertical_position = "up"
        elif y > image_height * 0.67:
            vertical_position = "down"
        else:
            vertical_position = "middle"

        # 크기 분류 (s, m, l)
        area = width * height
        size_category = "s" if area < 10000 else ("m" if area < 30000 else "l")

        return f"{horizontal_position} {vertical_position} {size_category}"

    # 📌 Text 위치 분석
    text_positions = [classify_position(td['x'], td['y'], td['width'], td['height']) for td in text_details]
    if not text_positions:  # 텍스트가 없을 경우
        text_positions = ["텍스트 없음"]

    # 📌 Person 위치 분석
    person_positions = [
        classify_position(obj['x'], obj['y'], obj['width'], obj['height'])
        for obj in object_details if obj['label'] == 'person'
    ]
    if not person_positions:  # 사람 객체가 없을 경우
        person_positions = ["사람 없음"]

    # 📌 신뢰도 높은 텍스트 추출 (probability ≥ 0.7)
    high_confidence_texts = [td["text"] for td in text_details if td.get("probability", 0) >= 0.7]
    prob_text = high_confidence_texts if high_confidence_texts else ["해당 없음"]

    return text_positions, person_positions, prob_text


# 🖼️ DataFrame에 새로운 컬럼 추가
df[["text_positions", "person_positions", "prob_text"]] = df.apply(
    lambda row: pd.Series(
        classify_positions(
            image_width=800,  # 이미지 가로 크기
            image_height=600,  # 이미지 세로 크기
            text_details=row["text_details"],
            object_details=row["objects_details"]
        )
    ),
    axis=1
)

In [125]:
objects_li=[]
for i in df['objects_details']:
    object_li=[]
    for j in i:
        object_li.append(j['label'])
    objects_li.append(object_li)

assert len(df) == len(objects_li), "Length of text_position_li does not match the number of rows in df"
df['contain_object'] = objects_li

del df['text_details']
del df['objects_details']

In [127]:
print(df.iloc[0])

title               ENG│코스트코 고기 2.5키로 소분하는 자취 일상. 대파꼬치전, 또띠아와퍼 만들고...
video_id                                                  uxDZyUHO9sE
published_date                                             2024-08-18
thumbnail_url        https://i.ytimg.com/vi/uxDZyUHO9sE/hqdefault.jpg
view_count                                                  1113043.0
like_count                                                    27876.0
comment_count                                                   597.0
category_id                                                        22
duration                                                     00:33:51
channel_id                       [planD플랜디, UCPaApoLkmPqxCeZF7gjoc3w]
brightness                                                  121.51537
contrast                                                    77.338087
dominant_colors     [(darkkhaki, 0.2679224537037037), (black, 0.25...
text_positions                                               [텍스트 없음]
person_positions    

## 채널 정보 크롤링

In [3]:
with open('channel_ids.json', "r", encoding="utf-8") as file:
        data = json.load(file)
len(data)

602

In [11]:
data=data[407:]

In [13]:
# 여러 개의 User-Agent 리스트
USER_AGENTS = [
    # Chrome (Windows)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",

    # Chrome (Mac)
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    
    # Chrome (Linux)
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0",
    
    # Firefox (Windows)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:115.0) Gecko/20100101 Firefox/115.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:102.0) Gecko/20100101 Firefox/102.0",
    
    # Firefox (Mac)
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11.6; rv:94.0) Gecko/20100101 Firefox/94.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; rv:102.0) Gecko/20100101 Firefox/102.0",

    # Safari (Mac)
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Version/16.2 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Version/15.1 Safari/537.36",

    # Edge (Windows)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.0.0",

    # Opera
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/97.0.4719.63",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/97.0.4719.63",
    
    # Android (Chrome)
    "Mozilla/5.0 (Linux; Android 12; SM-G998B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 11; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.101 Mobile Safari/537.36",
    
    # Android (Firefox)
    "Mozilla/5.0 (Android 12; Mobile; rv:109.0) Gecko/109.0 Firefox/109.0",
    "Mozilla/5.0 (Android 10; Mobile; rv:102.0) Gecko/102.0 Firefox/102.0",
    
    # iPhone (Safari)
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Mobile/15E148 Safari/604.1",
    
    # iPad (Safari)
    "Mozilla/5.0 (iPad; CPU OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 15_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Mobile/15E148 Safari/604.1",
]


# Chrome 옵션 설정
options = Options()
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")  # 랜덤 User-Agent 적용

# WebDriver 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

channel_name_li=[]
subscriber_count_li=[]
video_count_li=[]

for i in tqdm(data):
    try:
        # 크롤링하려는 YouTube 채널 URL
        channel_url = f'https://www.youtube.com/channel/{i[1]}'

        # 채널 페이지 열기
        driver.get(channel_url)

        # 페이지 로딩 대기 (동적 콘텐츠가 로딩될 시간을 줍니다)
        time.sleep(1)

        # 채널명 가져오기
        try:
            channel_name = driver.find_element(By.XPATH, '//meta[@property="og:title"]').get_attribute('content')
        except:
            channel_name = "N/A"
        channel_name_li.append(channel_name)

        # 구독자 수 가져오기
        try:
            subscriber_count = driver.find_element(
                By.XPATH, '//*[@id="page-header"]/yt-page-header-renderer/yt-page-header-view-model/div/div[1]/div/yt-content-metadata-view-model/div[2]/span[1]'
            ).text
        except:
            subscriber_count = "N/A"
        subscriber_count_li.append(subscriber_count)

        # 총 동영상 수 가져오기
        try:
            video_count = driver.find_element(
                By.XPATH, '//*[@id="page-header"]/yt-page-header-renderer/yt-page-header-view-model/div/div[1]/div/yt-content-metadata-view-model/div[2]/span[3]'
            ).text
        except:
            video_count = "N/A"
        video_count_li.append(video_count)

    except Exception as e:
        print(f"채널 {i[1]} 크롤링 중 오류 발생: {e}")
        continue  # 오류 발생 시 다음 채널로 이동

# 브라우저 종료
driver.quit()

100%|████████████████████████████████████████████████████████████████████████████████| 195/195 [29:29<00:00,  9.07s/it]


In [15]:
import pandas as pd
df_sub=pd.DataFrame({'channel_id':data,
                     'subscriber_count':[s.replace('구독자', '').replace('명', '').strip() for s in subscriber_count_li],
                     'total_video_count':[s.replace('동영상', '').replace('개', '').strip() for s in video_count_li]})
df_sub

,channel_id,subscriber_count,total_video_count
0,"[닥터셰프, UCc4GDo3-a9FyGd8_zGpYJZQ]",128,38
1,"[주부나라, UCzIOIStANWRgYa3BRuJOg8Q]",161만,1.7천
2,"[이 남자의 cook, UCy2WX3w5UyYFHBDHyWFKNUQ]",249만,987
3,"[KBS Entertain: 깔깔티비, UC271vX3c3kYZsgpz7-zPBoA]",106만,8.2천
4,"[유라야놀자(Let's play YURA), UCx8IhwapX8E7uooFYJIe...",227만,2천
...,...,...,...
190,"[탑텐사커, UC0Ue4AZLKcqDl2hqZn4eP4g]",1.76만,261
191,"[스포츠 스토리, UCVl5dHKhpf4Z07HIH_KbnWQ]",4.54만,308
192,"[Snooker Fan Zone, UC9nUNwRwWBoPZHNtRzDUp8g]",7.62천,73
193,"[초이스포츠, UCbcl1BPu7sLvaznemwlwu1A]",737,49


In [17]:
df_pre=pd.read_csv('channel_info.csv',encoding='utf-8-sig')
df_total=pd.concat([df_pre,df_sub]).reset_index(drop=True)
df_total

,channel_id,subscriber_count,total_video_count
0,"['보겸TV', 'UCu9BCtGIEr73LXZsKmoujKw']",1560만,9.3천
1,"['JYP Entertainment', 'UCaO6TYtlC8U5ttz62hTrZgg']",3040만,2.3천
2,"['늘이농', 'UCoFfV8sTi5PWKEdvJP2XzWQ']",37.5만,451
3,"['planD플랜디', 'UCPaApoLkmPqxCeZF7gjoc3w']",225만,256
4,"['JTBC Drama', 'UCkbJc8jMcTXwhtmN5VMwfXg']",493만,3.2만
...,...,...,...
597,"[탑텐사커, UC0Ue4AZLKcqDl2hqZn4eP4g]",1.76만,261
598,"[스포츠 스토리, UCVl5dHKhpf4Z07HIH_KbnWQ]",4.54만,308
599,"[Snooker Fan Zone, UC9nUNwRwWBoPZHNtRzDUp8g]",7.62천,73
600,"[초이스포츠, UCbcl1BPu7sLvaznemwlwu1A]",737,49


In [19]:
df_total.to_csv('channel_info.csv', index=False, encoding='utf-8-sig')

### 채널정보 칼럼 추가

In [141]:
df['channel_id'] = df['channel_id'].astype(str)
df_total['channel_id'] = df_total['channel_id'].astype(str)
df=pd.merge(left=df, right=df_total, how='inner', on='channel_id')
df

,title,video_id,published_date,thumbnail_url,view_count,...,person_positions,prob_text,contain_object,subscriber_count,total_video_count
0,"ENG│코스트코 고기 2.5키로 소분하는 자취 일상. 대파꼬치전, 또띠아와퍼 만들고...",uxDZyUHO9sE,2024-08-18,https://i.ytimg.com/vi/uxDZyUHO9sE/hqdefault.jpg,1113043.0,...,[middle middle l],[해당 없음],"[bottle, bottle, bottle, bottle, bowl, person,...",225만,256
1,"＂이불을 한 채만 쓸까요?＂ 더 가까워진 임지연♥추영우, 찐-한 합방💞 | 옥씨부인...",n9TTmAATM1M,2025-01-19,https://i.ytimg.com/vi/n9TTmAATM1M/hqdefault.jpg,1975060.0,...,[사람 없음],[해당 없음],[],493만,3.2만
2,더 이상 물러날 곳❌ 의금부에 추포된 하율리-최정우 | 옥씨부인전 16회 | JTB...,6L66YeMkkLQ,2025-01-26,https://i.ytimg.com/vi/6L66YeMkkLQ/hqdefault.jpg,1248961.0,...,"[middle middle l, middle middle l]",[해당 없음],"[person, person]",493만,3.2만
3,[스페셜 선공개] ＂현감 얼굴에 칼자국..＂ 임지연 향한 윤지혜의 의미심장한 질문....,Xu6MTsCNunA,2025-01-15,https://i.ytimg.com/vi/Xu6MTsCNunA/hqdefault.jpg,1093319.0,...,[middle middle l],[해당 없음],[person],493만,3.2만
4,"그렇게 유난을 떨던 공개 장기연애 끝, 한달 뒤 떠난 도쿄여행에서 생긴 일💧",ehUnAK-uNqk,2025-01-12,https://i.ytimg.com/vi/ehUnAK-uNqk/hqdefault.jpg,341820.0,...,[middle middle l],[해당 없음],[person],13.5만,85
5,"VLOG • 연휴동안 생긴 일✨ 자취요리, smp 리터치, 어색한 썸남보이랑 데이트...",ACdNDBZw_4c,2025-02-17,https://i.ytimg.com/vi/ACdNDBZw_4c/hqdefault.jpg,88353.0,...,"[middle middle l, left middle m, middle middle m]","[new!, 연휴의]","[person, person, cup, person]",13.5만,85
6,"VLOG • 두피문신 smp 후기👩🏻‍🦲, 레전드 야장맛집 뿌시기, 행궁동과 홍대...",eZVo_G-S_5g,2024-10-12,https://i.ytimg.com/vi/eZVo_G-S_5g/hqdefault.jpg,84381.0,...,"[middle middle l, left middle l, middle up s]",[해당 없음],"[person, cup, cup, person, person, cup]",13.5만,85
7,"VLOG • 그래요 나 연애해요,, 예뻐야 하는 날 야무지게 준비하고🧖🏻‍♀️ 남친...",qiuSUSiyGkI,2025-03-10,https://i.ytimg.com/vi/qiuSUSiyGkI/hqdefault.jpg,77976.0,...,"[middle middle l, middle up m, left up s, left...",[브이로그;],"[person, fork, person, person, dining table, c...",13.5만,85
8,"VLOG • 2달 차 솔로 일상😇 기어코 이사, 대전여행, 청첩장모임, 집밥, 행궁...",WXgiwBUPhyc,2025-01-24,https://i.ytimg.com/vi/WXgiwBUPhyc/hqdefault.jpg,58725.0,...,[left middle s],[해당 없음],"[person, bed, cup]",13.5만,85
9,"입냄새의 진짜 원인, 목에 낀 똥을 싹 없애는 간단한 방법 (이영섭 원장 2부)",dR5scpolbQY,2024-05-10,https://i.ytimg.com/vi/dR5scpolbQY/hqdefault.jpg,4179526.0,...,[middle middle l],"[(이비인무과 영업비일), 입냄새]",[person],266만,979


In [ ]:
def convert_subscriber_count(value):
    if '만' in value:
        return float(value.replace('만', '')) * 10000
    return float(value)

df['subscriber_count'] = df['subscriber_count'].apply(convert_subscriber_count)
df['total_video_count'] = df['total_video_count'].apply(convert_subscriber_count)

In [163]:
df.iloc[5,:]

title                VLOG • 연휴동안 생긴 일✨ 자취요리, smp 리터치, 어색한 썸남보이랑 데이트...
video_id                                                   ACdNDBZw_4c
published_date                                              2025-02-17
thumbnail_url         https://i.ytimg.com/vi/ACdNDBZw_4c/hqdefault.jpg
view_count                                                     88353.0
like_count                                                       898.0
comment_count                                                    114.0
category_id                                                         22
duration                                                      00:12:53
channel_id                    ['현주zutube', 'UCJMMSPO-0YnhxRWfGOPmqvA']
brightness                                                   52.929369
contrast                                                     57.947802
dominant_colors      [(black, 0.563136574074074), (darkslategrey, 0...
text_positions                              [middle up s, middle up s]
person